In [56]:
import os
import time
import requests
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML
from pytrends.request import TrendReq
from tqdm.notebook import tqdm
from voila.utils import wait_for_request
from ipywidgets import Button, widgets, Accordion

In [48]:
EXACT_KEYWORDS = []
TITLES_LIST = []
KEYWORDS_EXTENDED = []
TITLES_LIST_EXTENDED = []
pytrend = TrendReq()

In [49]:
uploader = widgets.FileUpload(
    accept='.xlsx',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
display(uploader)

FileUpload(value={}, accept='.xlsx', description='Upload')

In [50]:
checkboxer = widgets.Checkbox(
    value=False,
    description='Uwzględnij w analizie gry bez kategorii*',
    disabled=False,
    indent=False
)

In [51]:
display(checkboxer)

Checkbox(value=False, description='Uwzględnij w analizie gry bez kategorii*', indent=False)

In [54]:
def validate(uploaded_file):

    global EXACT_KEYWORDS
    global TITLES_LIST
    global KEYWORDS_EXTENDED
    global TITLES_LIST_EXTENDED
    global final_df_all_codes
    try:
        input_file = list(uploaded_file.value.values())[0]
        content = input_file['content']
        df_upload = pd.read_excel(content, header=None)
        df_upload = df_upload.replace(r'\n','', regex=True).replace(';','', regex=True).dropna()
        keywords = df_upload.iloc[:, 0].to_list()
        keywords = [k for k in keywords if k]

        keywords_codes = []
        all_codes = []
        for keyword in tqdm(keywords, leave=False):
            data = (pytrend.suggestions(keyword))
            for d in data:
                all_codes.append(d)
            dicti = next(
                (item for item in data if "game" in item["type"]),
                {
                    'mid': keyword,
                    'title': keyword,
                    'type': "0"
                }
            )
            if dicti is not None:
                keywords_codes.append(dicti)
            else:
                continue
                    
        df_codes = pd.DataFrame(keywords_codes)      #dataframe with single, most popular suggestion 
        df_all_codes = pd.DataFrame.from_records(all_codes) #dataframe with all keyword suggestions
        
        #delete suggestions not game related
        final_df_all_codes = df_all_codes.loc[df_all_codes["type"].str.contains("game")]
        final_df = df_codes.loc[df_codes["type"].str.contains("game")] 
        
        global WRONG_ID_DF
        WRONG_ID_DF = df_codes.loc[~df_codes["type"].str.contains("game")].drop(columns='mid') 
        #save suggestions not game related to another dataframe for further use
        
        #keep only the non overlapping keywords suggestions 
        joined_df = pd.concat([final_df_all_codes, final_df])
        joined_df = joined_df.drop_duplicates(keep=False)
        
        #delete keywords exact ID from display 
        df_more_keywords = joined_df.loc[ : , joined_df.columns != 'mid']
        df_display = final_df.loc[ : , final_df.columns != 'mid']
        
        df_default_keywords = pd.concat([df_display, WRONG_ID_DF])
        
        display(HTML(df_default_keywords.to_html(index=False)))

        
        WRONG_ID_DF['type'].apply(pd.to_numeric)
        WRONG_ID_DF['type'] = "0.00"
        WRONG_ID_DF.rename(
            columns={
                'type': 'średnia',
                'title': 'tytuł'
                },
            inplace=True
        )
        
        wrong_ID = WRONG_ID_DF['tytuł'].to_list()
        wrong_id_list = []
        for i in wrong_ID:
            wrong_id_list.append('"' + i + '"')
        
        WRONG_ID_DF = WRONG_ID_DF.set_index('tytuł')
        
        EXACT_KEYWORDS = final_df['mid'].to_list()
        TITLES_LIST = final_df['title'].to_list()
        
        KEYWORDS_EXTENDED = EXACT_KEYWORDS + wrong_id_list
        TITLES_LIST_EXTENDED = TITLES_LIST + wrong_id_list
        
        with output3:
            output3.clear_output()
            display(HTML(df_more_keywords.to_html(index=False)))
        
        display(acc)
        

    except IndexError:
        print("Błąd: nie przesłano pliku bądź przesłano pusty plik")
    except requests.exceptions.Timeout:
        print("Błąd: Google nie odpowiada. Spróbuj ponownie później")
    except:
        print("Coś poszło nie tak")


    
    

In [27]:
output3 = widgets.Output()


In [28]:
acc = Accordion(children=[output3], selected_index=None)
acc.set_title(0, 'pokaż więcej sugestii')

In [29]:
button2 = widgets.Button(description="wczytaj tytuły")
output2 = widgets.Output()

display(button2)


@output2.capture(clear_output=True)
def on_button_clicked_validate(b):
        validate(uploader)

button2.on_click(on_button_clicked_validate)

Button(description='wczytaj tytuły', style=ButtonStyle())

In [30]:
display(output2)

Output()

In [32]:
time_frame = widgets.Dropdown(
    options=[
        ("ostatni miesiąc", "today 1-m"),
        ("ostatnie 3 miesiące", "today 3-m"),
        ("ostatni rok", "today 12-m"),
        ("ostatnie 5 lat", "today 5-y"),
        ("od 2004-", "all"),
        ("własna data", "custom_date")
    ],
    value='today 12-m',
    description='Zakres czasu:',
    disabled=False,
)

In [33]:
custom_timeframe_start = widgets.DatePicker(
    description='Początek',
    disabled=False
)
custom_timeframe_stop = widgets.DatePicker(
    description='Koniec',
    disabled=False
)

In [34]:
location = widgets.Dropdown(
    options=[
        ('cały świat', ''),
        ('Stany Zjednoczone', 'US'),
        ('Polska', 'PL'),
        ('Wielka Brytania', 'GB'),
        ('Niemcy', 'DE'),
        ('Japonia', 'JP'),
        ('Rosja', 'RU'),
        ('Korea Południowa', 'KR')
    ], 
    value='',
    description='Region:',
    disabled=False,
)

In [35]:
display(custom_timeframe_start, custom_timeframe_stop)

DatePicker(value=None, description='Początek')

DatePicker(value=None, description='Koniec')

In [36]:

display(time_frame)

display(location)

Dropdown(description='Zakres czasu:', index=2, options=(('ostatni miesiąc', 'today 1-m'), ('ostatnie 3 miesiąc…

Dropdown(description='Region:', options=(('cały świat', ''), ('Stany Zjednoczone', 'US'), ('Polska', 'PL'), ('…

In [37]:
def compare(kw_list):

    trends = dict()
    try:
        for i in tqdm(kw_list, leave=False):
            time.sleep(1)
            if time_frame.value == "custom_date":
                zakres = str(custom_timeframe_start.value) + " " + str(custom_timeframe_stop.value)
                pytrend.build_payload(
                    [i, '/m/0c7kdf'],
                    cat=0,
                    timeframe=zakres,
                    geo=location.value
                )
            else:
                zakres = time_frame.value
                pytrend.build_payload(
                    [i, '/m/0c7kdf'],
                    cat=0,
                    timeframe=zakres,
                    geo=location.value
                )
            trends[i] = pytrend.interest_over_time()[i]

        trends_df = pd.DataFrame.from_dict(trends)
        trends_df.columns = TITLES_LIST

        av_column = trends_df.mean(axis=0, numeric_only=True).round(2).sort_values(ascending=False)
        av_column = av_column.to_frame(name="średnia")
        
        final_av_column = pd.concat([av_column, WRONG_ID_DF])

        final_av_column["średnia"] = final_av_column["średnia"].map(str)
        final_av_column["średnia"] = final_av_column["średnia"].str.replace(".", ",", regex=True)
        
        csv_filename = ("results_" + time.strftime('%Y-%m-%d %H:%M') + time_frame.label + ".csv")
        final_av_column.to_csv(csv_filename)
        
        display(HTML(f"<a download='{csv_filename}' href='../files/{csv_filename}'>"
                 "Kliknij tutaj aby pobrać raport</a>"))
        
        display(final_av_column)
        
        if time_frame.value == "custom_date":
            print(("zakres: " + zakres + ';'), ("region: " + location.label))
        else:
            print(("zakres: " + time_frame.label + ';'), ("region: " + location.label))
        
        print("tytułom bez kategorii automatycznie przypisano wartość 0")

    except requests.exceptions.Timeout:
        print("Błąd: Google nie odpowiada. Spróbuj ponownie później")
    except NameError:
        print("Błąd: nie przesłano pliku")
    except:
        print("Coś poszło nie tak")

In [38]:
def compare_all(kw_list):

    trends = dict()
    try:
        for i in tqdm(kw_list, leave=False):
            time.sleep(1)
            if time_frame.value == "custom_date":
                zakres = str(custom_timeframe_start.value) + " " + str(custom_timeframe_stop.value)
                pytrend.build_payload(
                    [i, '/m/0c7kdf'],
                    cat=0,
                    timeframe=zakres,
                    geo=location.value
                )
            else:
                zakres = time_frame.value
                pytrend.build_payload(
                    [i, '/m/0c7kdf'],
                    cat=0,
                    timeframe=zakres,
                    geo=location.value
                )
            trends[i] = pytrend.interest_over_time()[i]

        trends_df = pd.DataFrame.from_dict(trends)
        trends_df.columns = TITLES_LIST_EXTENDED

        final_av_column = trends_df.mean(axis=0, numeric_only=True).round(2).sort_values(ascending=False)
        final_av_column = final_av_column.to_frame(name="średnia")

        final_av_column["średnia"] = final_av_column["średnia"].map(str)
        final_av_column["średnia"] = final_av_column["średnia"].str.replace(".", ",", regex=True)
        
        csv_filename = ("results_" + time.strftime('%Y-%m-%d %H:%M') + time_frame.label + ".csv")
        final_av_column.to_csv(csv_filename)
        
        display(HTML(f"<a download='{csv_filename}' href='../files/{csv_filename}'>"
                 "Kliknij tutaj aby pobrać raport</a>"))
        
        display(final_av_column)
        
        if time_frame.value == "custom_date":
            print(("zakres: " + zakres + ';'), ("region: " + location.label))
        else:
            print(("zakres: " + time_frame.label + ';'), ("region: " + location.label))
            
        print("tytuły bez kategorii zostały ujęte w cudzysłów")

    except requests.exceptions.Timeout:
        print("Błąd: Google nie odpowiada. Spróbuj ponownie później")
    except NameError:
        print("Błąd: nie przesłano pliku")
    except:
        print("Coś poszło nie tak")

In [39]:
button = widgets.Button(description="wygeneruj")
output = widgets.Output()

display(button, output)


@output.capture(clear_output=True)
@output2.capture(clear_output=True)
def on_button_clicked(b):
    with output:
        if checkboxer.value is False:
            compare(EXACT_KEYWORDS)
        else:
            compare_all(KEYWORDS_EXTENDED)


button.on_click(on_button_clicked)

Button(description='wygeneruj', style=ButtonStyle())

Output()

In [57]:
wait_for_request()
query_string = os.getenv('QUERY_STRING')